In [1]:
import pandas as pd
import numpy as np

In [2]:
movies = pd.read_csv('moviesFinal.csv')

In [3]:
movies.head(1)

,imdb_title_id,title,year,genre,duration,country,language,director,writer,production_company,actors,description,avg_vote,budget
0,tt0035423,Kate & Leopold,2001,"Comedy,Fantasy,Romance",118,USA,"English, French",James Mangold,"Steven Rogers, James Mangold",Konrad Pictures,"Meg Ryan, Hugh Jackman, Liev Schreiber, Brecki...",An English Duke from 1876 is inadvertedly drag...,6.4,$ 48000000


In [4]:
print(movies[movies['imdb_title_id'] == 'tt0113497'])

     imdb_title_id    title  year                    genre  duration country  \
4737     tt0113497  Jumanji  1995  Adventure,Comedy,Family       104     USA   

             language      director                           writer  \
4737  English, French  Joe Johnston  Jonathan Hensleigh, Greg Taylor   

     production_company                                             actors  \
4737   TriStar Pictures  Robin Williams, Jonathan Hyde, Kirsten Dunst, ...   

                                            description  avg_vote      budget  
4737  When two kids find and play a magical board ga...       7.0  $ 65000000  


In [78]:
# Convert genre column into dummy variable columns
# first get all genres from database

genres = []
for index, row in movies.iterrows():
    for genre in row['genre'].replace(" ", "").split(','):
        if not genre in genres:
            genres.append(genre)

In [83]:
# create them as new columns

for genre in genres:
    movies[genre] = 0

In [89]:
# assign value depending on genre of movie

for index, row in movies.iterrows():
    for genre in row['genre'].replace(" ", "").split(','):
        movies.loc[index, genre] = 1

In [8]:
df = movies.copy()

In [ ]:
df = df.sample(2000)

In [9]:
df = df.drop(columns=['imdb_title_id'])
df = df.drop(columns=['title'])
df = df.drop(columns=['genre'])
df = df.drop(columns=['country'])
df = df.drop(columns=['language'])
df = df.drop(columns=['director'])
df = df.drop(columns=['writer'])
df = df.drop(columns=['production_company'])
df = df.drop(columns=['actors'])
df = df.drop(columns=['description'])
df = df.drop(columns=['budget'])



In [73]:
# DESCRIPTION BASED RECOMMENDATION

from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet

In [74]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [75]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')

In [77]:
df = movies.copy()

In [78]:
tfidf_matrix = tf.fit_transform(df['description'])

In [56]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [1]:
df = df.reset_index()
titles = df['title']
indices = pd.Series(df.index, index=df['title'])

NameError: name 'df' is not defined

In [74]:
def get_recommendations(movieId):
    title = movies[movies['imdb_title_id'] == ("tt" + str(movieId).zfill(7))].iloc[0]['title']
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[0:30]
    sim_scores_filtered = []
    for i in sim_scores:
        if i[1] > 0.05:
            sim_scores_filtered.append(i)
    movie_indices = [i[0] for i in sim_scores_filtered]
    print(sim_scores_filtered)
    recommends = pd.DataFrame()
    genres = movies[movies['imdb_title_id'] == ("tt" + str(movieId).zfill(7))].iloc[0]['genre'].split(',')
    imdb_score = movies[movies['imdb_title_id'] == ("tt" + str(movieId).zfill(7))].iloc[0]['avg_vote']
    
    for index, row in df.iloc[movie_indices].iterrows():
        count = 0
        for genre in row['genre'].split(','):
            if genre in genres:
                count += 1
        if count > 0:
            if(row['avg_vote'] > ( imdb_score - 4)):
                recommends = recommends.append(row)
    return recommends

In [75]:
print(df[df['title'] == 'Inception']['imdb_title_id'])

13437    tt1375666
Name: imdb_title_id, dtype: object


In [76]:
recommends = get_recommendations(137523)
for index, row in recommends.iterrows():
    print(row['title'])
    print(row['genre'])
    print(row['description'], end='\n\n')


[(5765, 1.0000000000000004), (18860, 0.1751700332762778), (11688, 0.15725217091265634), (10415, 0.13024205105880943), (8011, 0.11347788177618406), (15424, 0.10161523219292971), (9318, 0.09856903852236878), (10307, 0.09688174234965628), (15787, 0.0890451204469477), (18753, 0.08834135479060255), (10408, 0.08643335241467281), (10195, 0.08596890842697358), (8166, 0.08422377957470813), (19013, 0.07597884778553775), (7729, 0.07399637756452294), (14354, 0.07241141826904991), (8250, 0.07133962712268123), (13188, 0.06698809803363377), (11799, 0.06291947746231928), (17200, 0.05528831504420456), (3005, 0.05275586986772581), (2338, 0.05207174076672733), (7164, 0.05195264123001065), (6248, 0.05033514550107874)]
Fight Club
Drama
An insomniac office worker and a devil-may-care soapmaker form an underground fight club that evolves into something much, much more.

Never Back Down
Action,Drama,Sport
A frustrated and conflicted teenager arrives at a new high school to discover an underground fight club a

Series([], Name: imdb_title_id, dtype: object)


In [25]:
ratings = pd.read_csv('ratingsUp.csv')

In [26]:
ratings

,userId,movieId,rating
0,1,114709,4.0
1,1,113228,4.0
2,1,113277,4.0
3,1,114369,5.0
4,1,114814,5.0
...,...,...,...
100831,610,4972582,4.0
100832,610,4425200,5.0
100833,610,5052448,5.0
100834,610,3315342,5.0


In [63]:
def get_recommendations_by_userdata(movieId):
    likedSameMovie = ratings[ratings['movieId'] == movieId]['userId'].unique()
    print(likedSameMovie)
    likedMovies = pd.DataFrame()
    
    for uid in likedSameMovie:
        likedMovies = likedMovies.append(ratings[ratings['userId'] == uid])
        likedMovies = likedMovies[likedMovies['rating'] >3.5]
        
    likedMovies = likedMovies.groupby('movieId')['movieId'].count().sort_values(ascending=False)
    likedMovies = likedMovies.head(1000)
    withDescription = pd.DataFrame()
    
    for movieId in likedMovies.keys():
        withDescription = withDescription.append(movies[movies['imdb_title_id'] == ("tt" + str(movieId).zfill(7))])
    
    tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
    tfidf_matrix = tf.fit_transform(withDescription['description'])
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    withDescription = withDescription.reset_index()
    titles = withDescription['title']
    indices = pd.Series(withDescription.index, index=withDescription['title'])
    
    idx = indices[withDescription.iloc[0]['title']]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[0:10]
    movie_indices = [i[0] for i in sim_scores]
    
    print(sim_scores)
    
    for index, row in withDescription.iloc[movie_indices].iterrows():
        print(row['title'])
        print(row['genre'])
        print(row['description'], end='\n\n')

In [27]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate


In [155]:
reader = Reader()
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)


In [156]:
svd = SVD()

In [161]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [162]:
ratings[ratings['userId'] == 25]

,userId,movieId,rating
4012,25,109686,4.0
4013,25,76759,5.0
4014,25,108052,5.0
4015,25,82971,5.0
4016,25,120815,5.0
4017,25,133093,5.0
4018,25,172495,5.0
4019,25,120737,5.0
4020,25,167261,5.0
4021,25,167260,5.0


In [149]:
ids = ratings[ratings['userId'] == 25]['movieId']
for id in ids:
    print(movies[movies['imdb_title_id'] == ("tt" + str(id).zfill(7))]['title'])

4466    Dumb and Dumber
Name: title, dtype: object
2117    Star Wars
Name: title, dtype: object
4387    Schindler's List
Name: title, dtype: object
2572    Raiders of the Lost Ark
Name: title, dtype: object
5519    Saving Private Ryan
Name: title, dtype: object
5713    The Matrix
Name: title, dtype: object
6141    Gladiator
Name: title, dtype: object
5481    The Lord of the Rings: The Fellowship of the Ring
Name: title, dtype: object
6092    The Lord of the Rings: The Two Towers
Name: title, dtype: object
6091    The Lord of the Rings: The Return of the King
Name: title, dtype: object
9889    The Dark Knight
Name: title, dtype: object
8241    Iron Man
Name: title, dtype: object
11262    WALL·E
Name: title, dtype: object
8092    Inglourious Basterds
Name: title, dtype: object
11847    Up
Name: title, dtype: object
12251    Shutter Island
Name: title, dtype: object
13437    Inception
Name: title, dtype: object
13348    The Dark Knight Rises
Name: title, dtype: object
15949    The Imitati

In [181]:
svd.predict(25, 253474)

Prediction(uid=25, iid=253474, r_ui=None, est=4.752370961317385, details={'was_impossible': False})

In [179]:
print(movies[movies['imdb_title_id'] == ("tt" + str(2713180).zfill(7))]['title'])

6351    Monsters, Inc.
Name: title, dtype: object


In [3]:
df3 = movies.copy()

In [4]:
df3['keywords'] = ''

In [6]:
xd = pd.read_csv('keywordList.csv')

In [80]:
keywordList = []
for i, row in xd.iterrows():
    keywordList.append(row['keyword'].split(' ')[0])

In [82]:
keywordList = list(set(keywordList))


In [83]:
keywordList

['professor',
 'heroin',
 'sibling',
 'immigrant',
 'lgbt',
 'revenge',
 'orphanage',
 'kaiju',
 'wedding',
 'good',
 'adaptation',
 'biography',
 '1940s',
 'jealousy',
 'survival',
 'cowboy',
 'spacecraft',
 'dinosaur',
 'brazilian',
 'tattoo',
 'france',
 'history',
 'fisherman',
 'hitman',
 'music',
 'night',
 'divorce',
 'adultery',
 'nazi',
 'trauma',
 'infidelity',
 'ice',
 'kids',
 'romantic',
 'royalty',
 'pilot',
 'inheritance',
 'sexploitation',
 'house',
 'mexico',
 'missing',
 'prison',
 'apocalypse',
 'puerto',
 'graffiti',
 'grief',
 'low',
 'hostage',
 'sister',
 'mansion',
 'brothel',
 'food',
 'helicopter',
 '1990s',
 'cuba',
 'escape',
 'christianity',
 'avant',
 'holocaust',
 'finland',
 'chanchada',
 'android',
 'cabaret',
 'french',
 'ninja',
 'fire',
 'aftercreditsstinger',
 'partially',
 'radio',
 'photography',
 'short',
 'moon',
 'nasa',
 'illness',
 'secret',
 'india',
 'christian',
 'voodoo',
 'adolescence',
 'biker',
 'fan',
 'pornography',
 'madness',
 'res

In [72]:
print(df3.iloc[10044]['description'])

Sydney stand-over man Jack Barrett (Friels) wants to retire to the seaside.


In [19]:
for i, row in df3.iterrows():
    keys = ''
    for word in row['description'].lower().split(' '):
        if word in keywordList:
            keys += word + ' '
    df3.at[i, 'keywords'] = keys

In [33]:
tfidf_matrix3 = tf.fit_transform(df3['keywords'])

In [63]:
cosine_sim3 = linear_kernel(tfidf_matrix3, tfidf_matrix3)

In [39]:
titles3 = df3['title']
indices3 = pd.Series(df3.index, index=df3['title'])

In [27]:
tfidf_matrix3[15]

NameError: name 'tfidf_matrix3' is not defined

In [70]:
def get_recommendations_by_keywords(movieId):
    title = df3[df3['imdb_title_id'] == ("tt" + str(movieId).zfill(7))].iloc[0]['title']
    idx = indices3[title]
    sim_scores = list(enumerate(cosine_sim3[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[0:10]
    print(sim_scores)
    movie_indices = [i[0] for i in sim_scores]
    return df3.iloc[movie_indices]

In [67]:
get_recommendations_by_keywords(137523)

NameError: name 'get_recommendations_by_keywords' is not defined

In [11]:
from PyDictionary import PyDictionary
dictionary=PyDictionary()

In [68]:
print (dictionary.meaning("slavery"))

Error: The Following Error occured: list index out of range
None


In [49]:
from nltk.stem.snowball import SnowballStemmer


In [60]:
stemmer = SnowballStemmer('english')

In [66]:
stemmer.stem('revolutionary')

'revolutionari'